### Make the Cars Stop in a case of cyclist

In [118]:
import carla
import numpy as np
import cv2
import time
import sys
sys.path.append('F:\CARLA\Windows\CARLA_0.9.15\PythonAPI\carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev, interp1d
import math

In [119]:
client = carla.Client('localhost', 2000)
FIXED_DELTA_SEC = 0.04
world = client.get_world()
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = FIXED_DELTA_SEC  # 20 FPS at 0.05
world.apply_settings(settings)

blueprints = [bp for bp in world.get_blueprint_library().filter('*')]
spawn_points = world.get_map().get_spawn_points()

In [120]:
def carorigin():
    global vehicle, vehicle_start_point
    vehicle_bp = world.get_blueprint_library().filter('*mini*')
    vehicle_start_point = spawn_points[94]
    vehicle = world.try_spawn_actor(vehicle_bp[0], vehicle_start_point)
    for _ in range(20):  # wait for half a second
            print("Still Falling Car")
            world.tick()
            time.sleep(0.05)

In [121]:
def bicycleorigin():
        global bicycle
        bicycle_bp = world.get_blueprint_library().filter('*crossbike*')
        bicycle_start_point = spawn_points[99]

        bicycle = world.try_spawn_actor(bicycle_bp[0], bicycle_start_point)
        
        new_location = bicycle_start_point.location + carla.Location(y=18)
        new_rotation = carla.Rotation(pitch=0, yaw=bicycle_start_point.rotation.yaw + 0, roll=0)

        
        bicyclepos = carla.Transform(new_location, new_rotation)
        bicycle.set_transform(bicyclepos)
        for _ in range(20):  # wait for half a second
            print("MStill Falling - Bicycle")
            world.tick()
            time.sleep(0.05)


In [122]:
def destroy():#Destroying the existing things
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()

In [123]:
vehicle = None
bicycle = None
vehicle_start_point = None
destroy()
carorigin()
bicycleorigin()
print(bicycle)


Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
Still Falling Car
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
MStill Falling - Bicycle
Actor(id=107, type=vehicle.bh.crossbike)


In [124]:
#destroy()

In [125]:
def DrawPointsFor30Sec(world, spawn_points):
    drawn_points = []
    for index, waypoint in enumerate(spawn_points):
        # Draw a string with an ID at the location of each spawn point
        point_id = f'ID: {index}'
        point = world.debug.draw_string(
            waypoint.location,
            point_id,
            draw_shadow=False,
            color=carla.Color(r=255, g=255, b=255),
            life_time=30,  # Set to 0 to make it persist indefinitely
            persistent_lines=True
        )
        drawn_points.append(point)

DrawPointsFor30Sec(world, spawn_points)

## Car and Cyclist position
### Car Start: ID: 1
### Car End: ID: 30
### Bycicyle start: ID 108

In [126]:
PREFERRED_SPEED = 30 #km/h
SPEED_THRESHOLD = 2 #When we close to the required speed, it drops the throttle
MAX_STEER_DEGREES = 40

def maintain_speed(s: float) -> float:
    
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.8
    else:
        return 0.4

In [127]:
def update_control(speed, last_error):
        Kp = 0.3
        Ki = 0.0
        Kd = 0.1
        dt = settings.fixed_delta_seconds
        integral_error = 0.0
        max_speed = 28



        current_speed = speed
        speed_error = max_speed - current_speed
        integral_error += speed_error * dt
        derivative_error = (speed_error - last_error) / dt
        last_error = speed_error

        # PID computation
        control_output = Kp * speed_error + Ki * integral_error + Kd * derivative_error

        # Map control output to throttle and brake command
        if control_output > 0:
            throttle = min(control_output, 1.0)
            brake = 0.0
        else:
            throttle = 0.0
            brake = min(abs(control_output), 1.0)
        
        print(f"This is Throttle: {throttle}")
        print(f"This is Braking: {brake}")
        return throttle, brake, last_error

### Setup the Collision Sensor

In [128]:
def process_collision(event):
    # Extract collision data
    global collision_happened
    other_actor = event.other_actor
    impulse = event.normal_impulse
    collision_location = event.transform.location
    print(f"Collision with {other_actor.type_id}")
    print(f"Impulse: {impulse}")
    print(f"Location: {collision_location}")
    collision_happened = True

In [129]:
collision_detector_bp = world.get_blueprint_library().find('sensor.other.collision')
collision_sensor = world.spawn_actor(
        collision_detector_bp,
        carla.Transform(),
        attach_to=vehicle
    )
collision_sensor.listen(lambda event: process_collision(event))

In [130]:
from ultralytics import YOLO
#Setup Camera
CAMERA_POS_Z = 1.5 
CAMERA1_POS_X = 0
CAMERA2_POS_X = 1
CAMERA1_POS_Y = 0.5
CAMERA2_POS_Y = -0.5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')


rightcamera1_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA1_POS_X, y = CAMERA1_POS_Y), carla.Rotation(yaw=90))
rightcamera1 = world.spawn_actor(camera_bp,rightcamera1_init_trans,attach_to=vehicle)

rightcamera2_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA2_POS_X, y = CAMERA1_POS_Y), carla.Rotation(yaw=90))
rightcamera2 = world.spawn_actor(camera_bp,rightcamera2_init_trans,attach_to=vehicle)

frontcamera1_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA1_POS_X, y = CAMERA1_POS_Y))
frontcamera1 = world.spawn_actor(camera_bp,frontcamera1_init_trans,attach_to=vehicle)

frontcamera2_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA1_POS_X, y = CAMERA2_POS_Y))
frontcamera2 = world.spawn_actor(camera_bp,frontcamera2_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

rightcamera1_data = {'image': np.zeros((image_h,image_w,4))}
rightcamera2_data = {'image': np.zeros((image_h,image_w,4))}
frontcamera1_data = {'image': np.zeros((image_h,image_w,4))}
frontcamera2_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
rightcamera1.listen(lambda image: camera_callback(image,rightcamera1_data))
rightcamera2.listen(lambda image: camera_callback(image,rightcamera2_data))
frontcamera1.listen(lambda image: camera_callback(image,frontcamera1_data))
frontcamera2.listen(lambda image: camera_callback(image,frontcamera2_data))
model = YOLO("best.pt")

In [131]:
import math

def match_bicycles_between_left_right(bicycles_left: list, bicycles_right: list):
    image_w = 640  # Image width
    fov = 90  # Field of view in degrees
    baseline = 1  # Baseline distance in meters
    focal_length = image_w / (2 * math.tan(math.radians(fov / 2)))  # Focal length in pixels

    
    y_threshold = 20  # pixels, adjust based on image scale
    matched_bicycles_with_distances = []

    for left_bicycle in bicycles_left:
        left_x, left_y = left_bicycle
        closest_bicycle = None
        min_dist = float('inf')

        for right_bicycle in bicycles_right:
            right_x, right_y = right_bicycle
            # Check if the y-coordinates are similar
            if abs(left_y - right_y) < y_threshold:
                # Calculate the distance (disparity)
                dist = abs(left_x - right_x)
                if dist < min_dist:
                    min_dist = dist
                    closest_bicycle = right_bicycle

        # If a match was found, calculate depth and add to list
        if closest_bicycle:
            right_x, _ = closest_bicycle
            disparity = abs(left_x - right_x)
            depth = (focal_length * baseline) / disparity if disparity != 0 else float('inf')
            matched_bicycles_with_distances.append((left_bicycle, depth))

    return matched_bicycles_with_distances

In [ ]:
# Make the cyclist go
collision_happened = False
bicycle.apply_control(carla.VehicleControl(throttle=1))
last_error = 0.0

while True:
    v = vehicle.get_velocity()
    speed = round(3.6*math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    print(f"This is Speed: {speed}")
    throttle_data, brake_data, last_error = update_control(speed, last_error)
    if avg_distance < 5:
        brake = 1
        throttle = 0
    vehicle.apply_control(carla.VehicleControl(throttle = throttle_data, brake = brake_data))
    world.tick()

    rightframe1 = rightcamera1_data['image']
    rightframe2 = rightcamera2_data['image']
    frontframe1 = frontcamera1_data['image']
    frontframe2 = frontcamera2_data['image']
    try:
        rightframe1 = cv2.cvtColor(rightframe1, cv2.COLOR_BGRA2BGR)
        rightframe2 = cv2.cvtColor(rightframe2, cv2.COLOR_BGRA2BGR)
        frontframe1 = cv2.cvtColor(frontframe1, cv2.COLOR_BGRA2BGR)
        frontframe2 = cv2.cvtColor(frontframe2, cv2.COLOR_BGRA2BGR)
    except:
        pass
    results_right1 = model(rightframe1, verbose=False)
    results_right2 = model(rightframe2)
    
    results_front1 = model(frontframe1)
    results_front2 = model(frontframe2)

    bicycles_right1 = []
    bicycles_right2 = []
    bicycles_front1 = []
    bicycles_front2 = []



    for result in results_right1:
        for box in result.boxes:
            # Extract box coordinates and other details
            x1, y1, x2, y2 = box.xyxy[0]
            center_x = int((x1 + x2) / 2)  # x-center of the bicycle
            center_y = int((y1 + y2) / 2)  # y-center of the bicycle
            bicycles_right1.append((center_x, center_y))
            conf = box.conf[0]            # Confidence score
            cls = box.cls[0]
            cv2.rectangle(rightframe1, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            cv2.putText(rightframe1, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            #break
              # Bounding box coordinates
    
    for result in results_right2:
        for box2 in result.boxes:
            # Extract box coordinates and other details
            x1, y1, x2, y2 = box2.xyxy[0]
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            bicycles_right2.append((center_x, center_y))
            conf = box2.conf[0]            # Confidence score
            cls = box2.cls[0]

            cv2.rectangle(rightframe2, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            cv2.putText(rightframe2, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    for result in results_front1:
        for box2 in result.boxes:
            # Extract box coordinates and other details
            x1, y1, x2, y2 = box2.xyxy[0]
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            bicycles_front1.append((center_x, center_y))
            conf = box2.conf[0]            # Confidence score
            cls = box2.cls[0]

            cv2.rectangle(frontframe1, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            cv2.putText(frontframe1, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    for result in results_front2:
        for box2 in result.boxes:
            # Extract box coordinates and other details
            x1, y1, x2, y2 = box2.xyxy[0]
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            bicycles_front2.append((center_x, center_y))
            conf = box2.conf[0]            # Confidence score
            cls = box2.cls[0]

            cv2.rectangle(frontframe2, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            cv2.putText(frontframe2, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    matched_bicycles_with_distances_right = match_bicycles_between_left_right(bicycles_right1, bicycles_right2)
    matched_bicycles_with_distances_front = match_bicycles_between_left_right(bicycles_front1, bicycles_front2)
    # Display distance for each matched bicycle on the left frame
    
    distance_front = 0
    distance_right = 0


    for (left_bicycle, distance) in matched_bicycles_with_distances_right:
        left_x, left_y = left_bicycle
        distance_labelright = f"Distance (right camera): {distance:.2f}m"
        distance_right = distance
        cv2.putText(rightframe1, distance_labelright, (left_x, left_y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        print(distance_labelright)


    for (left_bicycle, distance) in matched_bicycles_with_distances_front:
        left_x, left_y = left_bicycle
        distance_front = distance
        distance_label = f"Distance (front camera): {distance:.2f}m"
        cv2.putText(frontframe1, distance_label, (left_x, left_y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        print(distance_label)

    if distance_front > 0 and distance_right > 0:
        min_distance = np.min([distance_front, distance_right])
        max_distance = np.max([distance_front, distance_right])

        avg_distance = (min_distance*0.7 + max_distance*0.3)
    elif distance_front == 0:
        avg_distance = distance_right
    elif distance_right == 0:
        avg_distance = distance_front
    else:
        avg_distance = 30

    cv2.imshow('Front1 camera',frontframe1)
    cv2.imshow('Front2 camera',frontframe2)

    if cv2.waitKey(1) == ord('q') or collision_happened:
        break
    #time.sleep(0.5)

    if collision_happened:
        destroy()
        bicycleorigin()
        carorigin()
        break
cv2.destroyAllWindows() 
    

This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 0.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 1.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 3.0
This is Throttle: 1.0
This is Braking: 0.0
This is Speed: 4.0
This is Throttle: 1.0
This is Braking: 0.0
This is 

Collision with vehicle.bh.crossbike
Impulse: Vector3D(x=-3.566501, y=1.452277, z=-1.595243)
Location: Location(x=-54.344273, y=28.021179, z=0.042818)
